In [52]:
import yt_dlp as ytdlp
import shutil
import json
import subprocess
import urllib.request
import urllib.parse
import urllib.error
from pathlib import Path
from typing import Optional, List, Tuple

def get_sponsorblock_segments(video_id: str, categories: List[str]) -> List[Tuple[float, float]]:
    """
    Query SponsorBlock API directly for specific categories.
    Returns a list of (start, end) tuples to REMOVE.
    """
    # FIX: Use urlencode to handle spaces and special characters in the JSON string
    base_url = "https://sponsor.ajay.app/api/skipSegments"
    params = {
        'videoID': video_id,
        'categories': json.dumps(categories)
    }
    query_string = urllib.parse.urlencode(params)
    api_url = f"{base_url}?{query_string}"
    
    try:
        with urllib.request.urlopen(api_url) as response:
            data = json.loads(response.read().decode())
            
        segments = []
        for segment in data:
            # segment['segment'] contains [start, end]
            start, end = segment['segment']
            category = segment['category']
            segments.append((start, end))
            print(f"[SponsorBlock] Found segment to remove: {start}s - {end}s ({category})")
            
        # Sort by start time
        segments.sort(key=lambda x: x[0])
        return segments
        
    except urllib.error.HTTPError as e:
        if e.code == 404:
            print("[SponsorBlock] No segments found for this video.")
            return []
        print(f"[SponsorBlock] API Error: {e}")
        return []
    except Exception as e:
        print(f"[SponsorBlock] Connection Error: {e}")
        return []

def invert_segments(total_duration: float, remove_segments: List[Tuple[float, float]]) -> List[Tuple[float, float]]:
    """
    Takes a list of segments to REMOVE and calculates the segments to KEEP.
    """
    keep_segments = []
    current_pos = 0.0
    
    for start, end in remove_segments:
        if start > current_pos:
            # Keep everything before this bad segment
            keep_segments.append((current_pos, start))
        
        # Move current position to the end of the bad segment
        # (Ensure we don't go backwards if segments overlap)
        current_pos = max(current_pos, end)
        
    # Check if there is valid audio left after the last cut
    if current_pos < total_duration:
        keep_segments.append((current_pos, total_duration))
        
    return keep_segments

def cut_audio_manually(input_path: Path, output_path: Path, keep_segments: List[Tuple[float, float]]):
    """
    Uses FFmpeg 'atrim' and 'concat' filters to stitch together the good parts.
    """
    if not keep_segments:
        print("[Processing] No cuts needed. Copying file.")
        shutil.copy(input_path, output_path)
        return

    # Build the complex filter graph
    # Example: [0:a]atrim=0:10,asetpts=PTS-STARTPTS[a0];...
    filter_parts = []
    input_labels = []
    
    for i, (start, end) in enumerate(keep_segments):
        label = f"a{i}"
        # atrim cuts the audio, asetpts resets the timestamp so they flow smoothly
        filter_parts.append(f"[0:a]atrim=start={start}:end={end},asetpts=PTS-STARTPTS[{label}]")
        input_labels.append(f"[{label}]")
    
    # Concatenate all parts
    concat_cmd = f"{''.join(input_labels)}concat=n={len(input_labels)}:v=0:a=1[outa]"
    full_filter = ";".join(filter_parts) + ";" + concat_cmd
    
    cmd = [
        "ffmpeg", "-y",
        "-i", str(input_path),
        "-filter_complex", full_filter,
        "-map", "[outa]",
        "-vn", # Drop video
        str(output_path)
    ]
    
    print(f"[Processing] Running Manual FFmpeg Cut ({len(keep_segments)} parts)...")
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

def download_and_cut(url: str, out_dir: str = ".", audio_format: str = "opus") -> Path:
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)
    
    print("--- 1. Downloading Original Audio ---")
    ydl_opts = {
        "outtmpl": str(out_path / "%(title)s.%(ext)s"),
        "noplaylist": True,
        "quiet": True,
        "format": "bestaudio/best",
        "overwrites": True, # Force overwrite to ensure fresh test
        "postprocessors": [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': audio_format,
            'preferredquality': '192',
        }],
    }
    
    with ytdlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        if isinstance(info, dict) and info.get("entries"):
            info = info["entries"][0]

        video_id = info['id']
        duration = info['duration']
        
        temp_path = Path(ydl.prepare_filename(info))
        base_name = temp_path.stem
        downloaded_file = out_path / f"{base_name}.{audio_format}"

    print(f"--- 2. Fetching Cuts for ID: {video_id} ---")
    # You can add 'intro', 'outro', 'interaction' here if you want more cuts
    categories = ['music_offtopic', 'intro', 'outro', 'selfpromo', 'interaction']
    bad_segments = get_sponsorblock_segments(video_id, categories)
    
    print("--- 3. Calculating Cuts ---")
    good_segments = invert_segments(duration, bad_segments)
    
    # If the segments cover the whole video (no cuts), we are done
    if len(good_segments) == 1 and good_segments[0][0] == 0 and good_segments[0][1] == duration:
        print("--- No cuts required. Done. ---")
        return downloaded_file
        
    print(f"--- 4. Applying Cuts (Segments: {len(good_segments)}) ---")
    final_file = out_path / f"{base_name}_cut.{audio_format}"
    
    try:
        cut_audio_manually(downloaded_file, final_file, good_segments)
        
        # Cleanup: Replace original with cut version
        downloaded_file.unlink()
        final_file.rename(downloaded_file)
        print(f"--- Success! Saved to: {downloaded_file} ---")
        return downloaded_file
        
    except subprocess.CalledProcessError as e:
        print("FFmpeg failed.")
        raise e

if __name__ == "__main__":
    test_url = "https://www.youtube.com/watch?v=EvNbcn7WfsE"
    try:
        download_and_cut(test_url)
    except Exception as e:
        print(f"Error: {e}")

--- 1. Downloading Original Audio ---


--- 2. Fetching Cuts for ID: EvNbcn7WfsE ---             
[SponsorBlock] Found segment to remove: 0s - 42.722s (music_offtopic)
[SponsorBlock] Found segment to remove: 258.7s - 261.821s (music_offtopic)
--- 3. Calculating Cuts ---
--- 4. Applying Cuts (Segments: 2) ---
[Processing] Running Manual FFmpeg Cut (2 parts)...
--- Success! Saved to: SSIO x K.I.Z - Ich Ich Ich Ich Ich (Official Video).opus ---


In [ ]:
path = download_and_cut("https://youtu.be/7pLyutp2iJU?si=bdK7S11bHCifsVWr", audio_format="opus")
print(path)

--- 1. Downloading Original Audio ---


--- 2. Fetching Cuts for ID: 7pLyutp2iJU ---             
[SponsorBlock] Found segment to remove: 189.273s - 220.661s (music_offtopic)
--- 3. Calculating Cuts ---
--- 4. Applying Cuts (Segments: 2) ---
[Processing] Running Manual FFmpeg Cut (2 parts)...
--- Success! Saved to: brad pitt's cousin feat xp (iphone music video).opus ---
brad pitt's cousin feat xp (iphone music video).opus


In [36]:
import os
import re
import base64
import requests
from typing import Optional, Dict, Any

def _get_spotify_access_token(client_id: Optional[str] = None, client_secret: Optional[str] = None) -> str:
    """
    Obtain a Spotify API access token using Client Credentials flow.
    If client_id / client_secret are not provided, they will be read from
    SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET environment variables.
    Raises ValueError on missing credentials or requests.HTTPError on failure.
    """
    client_id = client_id or os.getenv("SPOTIFY_CLIENT_ID")
    client_secret = client_secret or os.getenv("SPOTIFY_CLIENT_SECRET")
    if not client_id or not client_secret:
        raise ValueError("Spotify client_id and client_secret must be provided or set in environment variables.")
    
    auth = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    headers = {
        "Authorization": f"Basic {auth}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    resp = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
    resp.raise_for_status()
    token = resp.json().get("access_token")
    if not token:
        raise RuntimeError("Failed to obtain Spotify access token.")
    return token

def _extract_spotify_track_id(track_id_or_url: str) -> str:
    """
    Accepts a Spotify track id, a spotify:track:... URI, or an open.spotify.com track URL,
    and returns the bare track id.
    """
    # direct id (22 chars base62)
    if re.fullmatch(r"[A-Za-z0-9]{22}", track_id_or_url):
        return track_id_or_url

    # spotify:track:...
    m = re.search(r"spotify:track:([A-Za-z0-9]{22})", track_id_or_url)
    if m:
        return m.group(1)

    # open.spotify.com/track/{id}
    m = re.search(r"open\.spotify\.com/track/([A-Za-z0-9]{22})", track_id_or_url)
    if m:
        return m.group(1)

    # sometimes shared urls contain ?si=...
    m = re.search(r"track/([A-Za-z0-9]{22})", track_id_or_url)
    if m:
        return m.group(1)

    raise ValueError("Could not parse track id from input: provide a track id, spotify:track:... URI or open.spotify.com/track/... URL")

def get_spotify_audio_tags(
    track_id_or_url: str,
    client_id: Optional[str] = None,
    client_secret: Optional[str] = None,
    include_audio_features: bool = True,
    include_audio_analysis: bool = False,
) -> Dict[str, Any]:
    """
    Return Spotify metadata + simplified tags for a track.

    Requires the helper functions already defined in the notebook:
      - _extract_spotify_track_id(track_id_or_url) -> str
      - _get_spotify_access_token(client_id, client_secret) -> str

    Returned dict contains:
      - 'track' : raw /tracks/{id} JSON
      - 'audio_features' : raw /audio-features/{id} JSON or None
      - 'audio_analysis' : raw /audio-analysis/{id} JSON or None (if requested)
      - 'tags' : simplified mapping with these keys:
          Album Cover Art, Album, Artist, Title, Genre, Style / Subgenre,
          Label, BPM, Duration_ms, Duration_s, Publish Date, Release Date
    """
    track_id = _extract_spotify_track_id(track_id_or_url)
    token = _get_spotify_access_token(client_id, client_secret)
    headers = {"Authorization": f"Bearer {token}"}
    base = "https://api.spotify.com/v1"

    out: Dict[str, Any] = {}

    # 1) Track metadata
    t_resp = requests.get(f"{base}/tracks/{track_id}", headers=headers)
    t_resp.raise_for_status()
    track = t_resp.json()
    out["track"] = track

    # 2) Album (fetch full album for label/images/release_date)
    album_info = track.get("album") or {}
    album_id = album_info.get("id")
    if album_id:
        try:
            a_resp = requests.get(f"{base}/albums/{album_id}", headers=headers)
            if a_resp.status_code == 200:
                album_info = a_resp.json()
            else:
                # keep partial album object and log
                print(f"[Spotify] /albums/{album_id} returned {a_resp.status_code}: {a_resp.text}")
        except requests.RequestException as e:
            print(f"[Spotify] album fetch error: {e}")

    # 3) Primary artist genres
    genres: List[str] = []
    artists = track.get("artists") or []
    if artists:
        primary = artists[0]
        artist_id = primary.get("id")
        if artist_id:
            try:
                ar_resp = requests.get(f"{base}/artists/{artist_id}", headers=headers)
                if ar_resp.status_code == 200:
                    artist_info = ar_resp.json()
                    genres = artist_info.get("genres", []) or []
                else:
                    print(f"[Spotify] /artists/{artist_id} returned {ar_resp.status_code}: {ar_resp.text}")
            except requests.RequestException as e:
                print(f"[Spotify] artist fetch error: {e}")

    # 4) Audio features (BPM/tempo)
    audio_features = None
    if include_audio_features:
        try:
            af_resp = requests.get(f"{base}/audio-features/{track_id}", headers=headers)
            if af_resp.status_code == 200:
                audio_features = af_resp.json()
            else:
                # log and continue; some tracks/regions may return 403/404
                print(f"[Spotify] audio-features {track_id} returned {af_resp.status_code}: {af_resp.text}")
                audio_features = None
        except requests.RequestException as e:
            print(f"[Spotify] audio-features request error: {e}")
            audio_features = None

    # 5) Audio analysis (optional)
    if include_audio_analysis:
        try:
            aa_resp = requests.get(f"{base}/audio-analysis/{track_id}", headers=headers)
            if aa_resp.status_code == 200:
                out["audio_analysis"] = aa_resp.json()
            else:
                print(f"[Spotify] audio-analysis {track_id} returned {aa_resp.status_code}: {aa_resp.text}")
                out["audio_analysis"] = None
        except requests.RequestException as e:
            print(f"[Spotify] audio-analysis request error: {e}")
            out["audio_analysis"] = None

    out["audio_features"] = audio_features

    # 6) Build simplified tags requested by user
    # Album cover art (prefer largest)
    images = (album_info.get("images") if isinstance(album_info, dict) else []) or []
    cover_art = images[0].get("url") if images else None

    album_name = album_info.get("name") if isinstance(album_info, dict) else album_info.get("name")
    artist_names = [a.get("name") for a in artists if a.get("name")]
    artist_joined = ", ".join(artist_names) if artist_names else None
    title = track.get("name")

    genre = genres[0] if genres else None
    style_subgenre = genres[1] if len(genres) > 1 else None

    label = album_info.get("label") if isinstance(album_info, dict) else None

    bpm = None
    if audio_features:
        bpm = audio_features.get("tempo")

    duration_ms = track.get("duration_ms")
    duration_s = round(duration_ms / 1000.0, 3) if isinstance(duration_ms, (int, float)) else None

    # Spotify exposes release_date on album object
    release_date = None
    if isinstance(album_info, dict):
        release_date = album_info.get("release_date") or release_date
    if not release_date:
        # fallback to track.album.release_date if present
        release_date = track.get("album", {}).get("release_date")

    publish_date = release_date  # for this use-case publish == release

    tags: Dict[str, Any] = {
        "Album Cover Art": cover_art,
        "Album": album_name,
        "Artist": artist_joined,
        "Title": title,
        "Genre": genre,
        "Style / Subgenre": style_subgenre,
        "Label": label,
        "BPM": bpm,
        "Duration_ms": duration_ms,
        "Duration_s": duration_s,
        "Publish Date": publish_date,
        "Release Date": release_date,
    }

    out["tags"] = tags
    return out

In [ ]:
import difflib
import unicodedata
import string

def parse_artist_title_from_filename(filename: str) -> (str, str):
    """
    Try to extract (artist, title) from a filename using common patterns.
    Examples it handles:
      - "Artist - Title (extra).opus"
      - "Artist — Title.ext"
      - "Title - Artist.ext" (less reliable)
    Returns (artist, title) — some values may be empty strings if parsing fails.
    """
    name = Path(filename).stem  # remove extension
    # Remove common tags in parentheses or brackets (years, quality, "Official Video", etc.)
    name = re.sub(r"\s*[\(\[][^\)\]]*[\)\]]\s*", " ", name).strip()
    # Try common separators
    for sep in [" - ", " — ", " – ", " — ", "-", "–", "—", ":"]:
        if sep in name:
            parts = [p.strip() for p in name.split(sep) if p.strip()]
            if len(parts) >= 2:
                # Heuristic: first is artist, rest joined is title
                artist = parts[0]
                title = " - ".join(parts[1:]) if len(parts) > 2 else parts[1]
                return artist, title
    # Fallback: sometimes filename is "Artist Title" or just "Title"
    # Try splitting on " — " or the last dash as a fallback
    m = re.match(r"^(?P<artist>[^-]+)-(?P<title>.+)$", name)
    if m:
        return m.group("artist").strip(), m.group("title").strip()
    # If nothing, treat whole as title
    return "", name

def _normalize_text(s: str) -> str:
    s = s or ""
    s = unicodedata.normalize("NFKD", s)
    s = s.lower()
    # remove punctuation
    s = s.translate(str.maketrans("", "", string.punctuation))
    # collapse whitespace
    s = " ".join(s.split())
    return s

def _best_match_in_album(album_id: str, target_title: str, token: str, market: str = "US") -> Optional[str]:
    """
    Fetch album tracks and return the track id that best matches target_title.
    Returns None if no tracks.
    """
    import requests
    headers = {"Authorization": f"Bearer {token}"}
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    params = {"limit": 50, "market": market}
    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    items = resp.json().get("items", [])
    if not items:
        return None

    norm_target = _normalize_text(target_title)
    best_id = None
    best_score = 0.0
    for it in items:
        name = it.get("name", "")
        norm_name = _normalize_text(name)
        score = difflib.SequenceMatcher(None, norm_target, norm_name).ratio()
        if score > best_score:
            best_score = score
            best_id = it.get("id")
    return best_id

def find_spotify_track_by_filename(filename: str, client_id: Optional[str] = None, client_secret: Optional[str] = None, market: str = "US") -> Optional[Dict[str, Any]]:
    """
    Parse artist/title from filename, perform a Spotify Search, and return either:
      - a full track dict (if available), or
      - the chosen track id string (if we must fetch by id later).
    Handles the case where the top search result is an album by fetching album tracks.
    """
    artist, title = parse_artist_title_from_filename(filename)
    # Build a query — prefer "track:title artist:artist" when possible
    if artist and title:
        q = f'track:"{title}" artist:"{artist}"'
    elif title:
        q = f'track:"{title}"'
    else:
        q = Path(filename).stem

    token = _get_spotify_access_token(client_id, client_secret)
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "q": q,
        "type": "track",
        "limit": 5,
        "market": market
    }

    import requests
    resp = requests.get("https://api.spotify.com/v1/search", headers=headers, params=params)
    try:
        resp.raise_for_status()
    except requests.HTTPError:
        return None

    data = resp.json()
    tracks = data.get("tracks", {}).get("items", [])
    # If we unexpectedly got zero tracks back from the 'track' search, try a looser query
    if not tracks:
        if title:
            params["q"] = f'track:"{title}"'
            resp = requests.get("https://api.spotify.com/v1/search", headers=headers, params=params)
            try:
                resp.raise_for_status()
            except requests.HTTPError:
                return None
            tracks = resp.json().get("tracks", {}).get("items", [])

    if not tracks:
        return None

    top = tracks[0]
    # Defensive: if the returned item is not a track (rare), handle album fallback
    item_type = top.get("type")
    if item_type == "track":
        return top  # full track object
    if item_type == "album":
        album_id = top.get("id")
        # pick best matching track from album using the parsed title
        chosen_track_id = _best_match_in_album(album_id, title or Path(filename).stem, token, market=market)
        if chosen_track_id:
            # return track id (caller will fetch features)
            return {"id": chosen_track_id}
        return None

    # other types: be defensive and try to return an id if present
    if "id" in top:
        return top
    return None

def get_spotify_tags_from_filename(filename: str, client_id: Optional[str] = None, client_secret: Optional[str] = None, include_audio_features: bool = False, include_audio_analysis: bool = False) -> Optional[Dict[str, Any]]:
    """
    High-level helper: find the track via filename, then return the same structure as get_spotify_audio_tags.
    Accepts either a track dict or a track id returned from `find_spotify_track_by_filename`.
    """
    track_or_id = find_spotify_track_by_filename(filename, client_id=client_id, client_secret=client_secret)
    if not track_or_id:
        print(f"[Spotify Search] No match found for filename: {filename}")
        return None

    # If we received a dict with an 'id' and 'name' it's likely a full track dict; if dict with only 'id' treat as id
    if isinstance(track_or_id, dict):
        track_id = track_or_id.get("id")
    else:
        track_id = track_or_id

    if not track_id:
        print("[Spotify Search] No track id available from search result.")
        return None

    return get_spotify_audio_tags(track_id, client_id=client_id, client_secret=client_secret, include_audio_features=include_audio_features, include_audio_analysis=include_audio_analysis)

In [39]:
fn = "Ryan Gosling - I'm Just Ken (From Barbie The Album) [Official Audio].opus"
tags = get_spotify_tags_from_filename(fn)
tags = tags["tags"]
print(tags)

3zXIvb4nZ3cTdT8CsbTy3U
{'Album Cover Art': 'https://i.scdn.co/image/ab67616d0000b273d16ac0ec27653182d719ab36', 'Album': 'Barbie The Album', 'Artist': 'Ryan Gosling', 'Title': "I'm Just Ken", 'Genre': None, 'Style / Subgenre': None, 'Label': 'Atlantic Records', 'BPM': None, 'Duration_ms': 222633, 'Duration_s': 222.633, 'Publish Date': '2023-07-21', 'Release Date': '2023-07-21'}


In [56]:
import urllib.request
import base64
import re
from pathlib import Path
from typing import Dict, Any

# pip install mutagen
import mutagen
from mutagen.oggopus import OggOpus
from mutagen.flac import Picture

_windows_illegal_re = re.compile(r'[<>:"/\\|?*\x00-\x1F]')

def _sanitize_filename(s: str, max_len: int = 200) -> str:
    if not s:
        return ""
    s = str(s)
    s = s.strip()
    s = _windows_illegal_re.sub("", s)
    s = re.sub(r"\s+", " ", s)
    if len(s) > max_len:
        s = s[:max_len].rstrip()
    return s

def _unique_path(path: Path) -> Path:
    """If path exists, append (1), (2), ... to make a unique filename."""
    if not path.exists():
        return path
    parent = path.parent
    stem = path.stem
    suffix = path.suffix
    i = 1
    while True:
        candidate = parent / f"{stem} ({i}){suffix}"
        if not candidate.exists():
            return candidate
        i += 1

def set_audio_tags(input_path: str | Path, tags: Dict[str, Any], overwrite: bool = True) -> Path:
    """
    Set metadata tags on an audio file using Mutagen (specifically for Opus/Ogg).
    After saving tags, rename the file to '{artists} - {title}{ext}' (sanitized).
    Returns the Path to the final file.
    """
    input_path = Path(input_path)
    if not input_path.exists():
        raise FileNotFoundError(f"{input_path} does not exist")

    # If we are not overwriting, copy the file to the new name first
    if not overwrite:
        out_path = input_path.with_name(input_path.stem + "_tagged" + input_path.suffix)
        import shutil
        shutil.copy2(input_path, out_path)
        target_path = out_path
    else:
        target_path = input_path
        out_path = input_path

    try:
        # Load the file with Mutagen
        audio = OggOpus(target_path)
        
        # 1. Handle Text Tags
        # Map friendly names to Vorbis comments
        key_map = {
            "Title": "title",
            "Artist": "artist",
            "Album": "album",
            "Genre": "genre",
            "Label": "organization", # 'publisher' often maps to organization in Vorbis
            "Date": "date",
            "Release Date": "date",
            "Description": "description",
        }

        for friendly, vorbis_key in key_map.items():
            val = tags.get(friendly)
            if val:
                audio[vorbis_key] = str(val)

        # 2. Handle Cover Art (The complex part for Opus)
        cover_url = tags.get("Album Cover Art")
        if cover_url:
            try:
                # Download image into memory
                req = urllib.request.Request(cover_url, headers={'User-Agent': 'Mozilla/5.0'})
                with urllib.request.urlopen(req) as response:
                    image_data = response.read()

                # Create a FLAC Picture block (Opus uses the FLAC picture structure)
                pic = Picture()
                pic.type = 3  # 3 = Front Cover
                # Try to guess mime from URL; fallback to jpeg
                if isinstance(cover_url, str) and cover_url.lower().endswith(('png',)):
                    pic.mime = "image/png"
                else:
                    pic.mime = "image/jpeg"
                pic.desc = "Cover Art"
                pic.data = image_data

                # Encode picture block to Base64 and set as metadata
                pic_data = pic.write()
                encoded_data = base64.b64encode(pic_data).decode("ascii")
                audio["metadata_block_picture"] = [encoded_data]
                
            except Exception as e:
                print(f"[Tagging] Failed to embed cover art: {e}")

        # Save changes
        audio.save()

        # --- Rename file to "{artists} - {title}{ext}" ---
        # Prefer tags provided, fallback to existing filename
        artists = tags.get("Artist") or tags.get("artist") or ""
        title = tags.get("Title") or tags.get("title") or input_path.stem

        artists_s = _sanitize_filename(artists)
        title_s = _sanitize_filename(title)

        if artists_s and title_s:
            new_stem = f"{artists_s} - {title_s}"
        elif title_s:
            new_stem = title_s
        elif artists_s:
            new_stem = artists_s
        else:
            new_stem = input_path.stem

        new_name = new_stem + input_path.suffix
        new_path = target_path.with_name(new_name)

        # Avoid overwriting existing files unintentionally
        new_path = _unique_path(new_path)

        try:
            target_path.rename(new_path)
            final_path = new_path
        except Exception as e:
            print(f"[Tagging] Failed to rename {target_path} -> {new_path}: {e}")
            final_path = target_path

        return final_path

    except Exception as e:
        print(f"[Tagging] Critical error processing {input_path}: {e}")
        raise e

In [50]:
path = "C:\\Users\\larsk\\Documents\\code\\youtube-to-navidrome\\Ryan Gosling Performs ＂I'm Just Ken＂ ｜ Barbie ｜ HBO Max.opus"
tags = tags
set_audio_tags(path, tags, overwrite=True)

WindowsPath("C:/Users/larsk/Documents/code/youtube-to-navidrome/Ryan Gosling Performs ＂I'm Just Ken＂ ｜ Barbie ｜ HBO Max.opus")

In [2]:
## Final tryout
path = download_and_cut("https://www.youtube.com/watch?v=EvNbcn7WfsE", audio_format="opus")
print(path)
tags = get_spotify_tags_from_filename(path)
set_audio_tags(path, tags["tags"], overwrite=True)

NameError: name 'download_and_cut' is not defined